In [1]:
import numpy as np
import h5py
import pandas as pd
from scipy.linalg import sqrtm

from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, AveragePooling1D, Input, Flatten, Dropout, LeakyReLU
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L2
import tensorflow as tf

import gc

#tf.random.set_seed(59843)
#np.random.seed(59843)

tf.random.set_seed(42)
np.random.seed(42)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2022-08-25 10:33:25.262286: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-25 10:33:26.466772: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-25 10:33:26.501929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 10:33:26.502240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 245.91GiB/s
2022-08-25 10:33:26.502271: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-25 10:33:26.523729: I tensorflow/stream_executor/pla

# Load data

In [2]:
def load_exps_and_labels(detections):
    all_exps, all_labels = [], []

    counter = 0

    for exp_name, data in detections.groupby('Exp'):
        counter += 1

        try:
            file = h5py.File(f'../data/real/{exp_name}.h5', 'r')
        except:
            print(f'Couldnt load exp {exp_name}')
            continue

        exp = []
        
        for channel_id in range(1, 17):
            channel = file[f'Chn{channel_id}']
            exp.append(channel)
        
        exp = np.swapaxes(exp, 1, 0)

        chirps_ids = []

        for _, (frame_id, depth, azi) in data[['Frame', 'Depth', 'Azi']].iterrows():
            chirps_ids = chirps_ids + list(np.linspace(frame_id*127, frame_id*128+127, 128, dtype=np.uint16))
            all_labels = all_labels + list(np.repeat(depth, 128))
            #chirps_ids.append(int(frame_id*127 + 63))
            #all_labels.append(depth)

        exp = exp[chirps_ids]

        all_exps = all_exps + list(exp)

        #if counter > 2:
        #    break
    
    all_exps = np.array(all_exps)
    all_labels = np.array([all_labels]).T
    #all_exps = np.swapaxes(all_exps, 1, 0)
    all_exps[:, :, :50] = 0
    
    return all_exps.astype(np.float32), all_labels.astype(np.float32)
        

In [3]:
all_detections = pd.read_csv('../data/real/TABULAR_CSV_ALL_CLASSES.csv')

In [4]:
detections = all_detections[all_detections['Exp'].str.contains('EXP_17_')].query('Class == "Motorcycle" and Depth <= 25')

In [5]:
exps_train = ['EXP_17_A', 'EXP_17_B', 'EXP_17_C', 'EXP_17_D', 'EXP_17_E', 'EXP_17_F', 'EXP_17_G', 'EXP_17_H', 'EXP_17_I']

detections_train = detections.query('Exp in @exps_train')
np.unique(detections_train['Exp'])


array(['EXP_17_B', 'EXP_17_C', 'EXP_17_D', 'EXP_17_E', 'EXP_17_F',
       'EXP_17_G', 'EXP_17_H', 'EXP_17_I'], dtype=object)

In [6]:
exps_val = ['EXP_17_M', 'EXP_17_Q', 'EXP_17_L']
#exps_val = ['EXP_17_M']

detections_val = detections.query('Exp in @exps_val')
np.unique(detections_val['Exp'])


array(['EXP_17_L', 'EXP_17_M', 'EXP_17_Q'], dtype=object)

In [7]:
exps_val2 = ['EXP_17_N', 'EXP_17_P', 'EXP_17_R']
#exps_val2 = ['EXP_17_R']

detections_val2 = detections.query('Exp in @exps_val2')
np.unique(detections_val2['Exp'])


array(['EXP_17_P', 'EXP_17_R'], dtype=object)

In [8]:
X_train, y_train = load_exps_and_labels(detections_train)
X_val, y_val = load_exps_and_labels(detections_val)
X_val2, y_val2 = load_exps_and_labels(detections_val2)

In [9]:
X_train.shape

(48256, 16, 1024)

In [10]:
X_val.shape

(12416, 16, 1024)

In [11]:
X_val2.shape

(15616, 16, 1024)

# Preprocess data

## Swap axis

In [12]:
X_train = np.swapaxes(X_train, 2, 1)
X_val = np.swapaxes(X_val, 2, 1)
X_val2 = np.swapaxes(X_val2, 2, 1)

## Scale

In [13]:
X_mean = 183.12559509277344
X_std = 973.3739013671875

In [14]:
X_train = np.subtract(X_train, X_mean, out=X_train)
X_train = np.divide(X_train, X_std, out=X_train)

In [15]:
X_val = np.subtract(X_val, X_mean, out=X_val)
X_val = np.divide(X_val, X_std, out=X_val)

In [16]:
X_val2 = np.subtract(X_val2, X_mean, out=X_val2)
X_val2 = np.divide(X_val2, X_std, out=X_val2)

## Shuffle

In [17]:
random_ids = np.arange(len(X_train))
np.random.shuffle(random_ids)

X_train[:, :, :] = X_train[random_ids]
y_train[:, :] = y_train[random_ids]

# Build Model

In [18]:
model = Sequential([
    Input((1024, 16)),
    Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=L2(0.5)),
    AveragePooling1D(strides=4),
    Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=L2(0.5)),
    AveragePooling1D(strides=4),
    Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=L2(0.5)),
    AveragePooling1D(strides=4),
    Flatten(),
    Dense(1, activation='linear', kernel_regularizer=L2(0.5))
])

2022-08-25 10:33:48.880475: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-25 10:33:48.910415: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2599990000 Hz
2022-08-25 10:33:48.911034: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ee191e0960 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-25 10:33:48.911051: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-25 10:33:48.979249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1022, 32)          1568      
_________________________________________________________________
average_pooling1d (AveragePo (None, 256, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 254, 64)           6208      
_________________________________________________________________
average_pooling1d_1 (Average (None, 64, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 128)           24704     
_________________________________________________________________
average_pooling1d_2 (Average (None, 16, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [20]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=[RootMeanSquaredError()])

# Train Model

In [21]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=5,
                    batch_size=128)

2022-08-25 10:33:51.457480: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3162505216 exceeds 10% of free system memory.


Epoch 1/5


2022-08-25 10:33:53.991261: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-25 10:33:54.302434: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-08-25 10:33:55.628499: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2022-08-25 10:33:55.781252: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-08-25 10:33:59.815961: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 3.34G (3586195456 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-08-25 10:33:59.815992: W tensorflow/core/common_runtime/bfc_allocator.cc:312] Garbage collection: deallocate free memory regions (i.e.,

374/377 [============================>.] - ETA: 0s - loss: 64.6959 - root_mean_squared_error: 6.4575

2022-08-25 10:34:09.592987: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 813694976 exceeds 10% of free system memory.


377/377 [==============================] - 6s 16ms/step - loss: 64.5220 - root_mean_squared_error: 6.4525 - val_loss: 55.2353 - val_root_mean_squared_error: 6.7857
Epoch 2/5
377/377 [==============================] - 5s 13ms/step - loss: 41.4423 - root_mean_squared_error: 5.7635 - val_loss: 31.4088 - val_root_mean_squared_error: 4.8064
Epoch 3/5
377/377 [==============================] - 4s 12ms/step - loss: 33.9936 - root_mean_squared_error: 5.0184 - val_loss: 30.5574 - val_root_mean_squared_error: 4.6524
Epoch 4/5
377/377 [==============================] - 5s 12ms/step - loss: 31.3834 - root_mean_squared_error: 4.7112 - val_loss: 27.6166 - val_root_mean_squared_error: 4.2758
Epoch 5/5
377/377 [==============================] - 5s 12ms/step - loss: 30.6520 - root_mean_squared_error: 4.6167 - val_loss: 26.0922 - val_root_mean_squared_error: 4.0646


In [22]:
del X_train
del y_train
gc.collect()

1476

# FID Socore

In [23]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	
	return fid

In [24]:
model.pop()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1022, 32)          1568      
_________________________________________________________________
average_pooling1d (AveragePo (None, 256, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 254, 64)           6208      
_________________________________________________________________
average_pooling1d_1 (Average (None, 64, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 128)           24704     
_________________________________________________________________
average_pooling1d_2 (Average (None, 16, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

## Evaluation 1

In [25]:
X_val2.shape
#np.random.shuffle(X_val)

(15616, 1024, 16)

In [43]:
calculate_fid(model, X_val2[:10000], X_val[:10000])

3.3107304181197956

In [46]:
noise = np.random.randn(10000, 1024, 16)
calculate_fid(model, X_val[:10000], noise)

66.37212836771631

: 

In [38]:
generations = np.load('../data/generated/conditional-nooutliers-1-15616samples.npy')
generations = np.swapaxes(generations, 2, 1)

In [39]:
generations.shape

(15616, 1024, 16)

In [40]:
generations_scaled = np.subtract(generations, X_mean, out=generations)
generations_scaled = np.divide(generations_scaled, X_std, out=generations_scaled)
np.random.shuffle(generations_scaled)

In [44]:
calculate_fid(model, X_val2[:10000], generations_scaled[:10000])

3.276240066979539

In [32]:
exp_17_m = np.load('../data/preprocessed/EXP_17_M_chirps.npy')
exp_17_m = np.swapaxes(exp_17_m, 2, 1)

In [33]:
exp_17_m.shape

(6144, 1024, 16)

In [34]:
exp_17_m_scaled = np.subtract(exp_17_m, X_mean, out=exp_17_m)
exp_17_m_scaled = np.divide(exp_17_m_scaled, X_std, out=exp_17_m_scaled)
np.random.shuffle(exp_17_m_scaled)

In [35]:
calculate_fid(model, exp_17_m_scaled[:3000], exp_17_m_scaled[3000:6000])

0.010722588448832504

In [45]:
calculate_fid(model, exp_17_m_scaled[:6000], generations_scaled[:6000])

1.1979097538933017

## Evaluation 2

In [71]:
generations = np.load('../data/generated/conditional-nooutliers-1-3000samples.npy')
generations = np.swapaxes(generations, 2, 1)

In [72]:
generations.shape, generations.mean(), generations.std(), generations.min(), generations.max()

((3000, 1024, 16), 183.87898, 985.5411, -3372.0, 4300.0)

In [67]:
generations_scaled = np.subtract(generations, X_mean, out=generations)
generations_scaled = np.divide(generations_scaled, X_std, out=generations_scaled)
np.random.shuffle(generations_scaled)

In [74]:
train = np.load('../data/preprocessed/EXP_17_M_chirps.npy')
train = np.swapaxes(train, 2, 1)

In [75]:
train.shape, train.mean(), train.std(), train.min(), train.max()

((6144, 1024, 16), 183.6147, 1001.003, -3884.0, 4772.0)

In [42]:
train_scaled = np.subtract(train, X_mean, out=train)
train_scaled = np.divide(train_scaled, X_std, out=train_scaled)
np.random.shuffle(train_scaled)

In [76]:
test = np.load('../data/preprocessed/EXP_17_R_chirps.npy')
test = np.swapaxes(test, 2, 1)

In [77]:
test.shape, test.mean(), test.std(), test.min(), test.max()

((9984, 1024, 16), 176.87022, 950.60364, -3056.0, 3412.0)

In [45]:
test_scaled = np.subtract(test, X_mean, out=test)
test_scaled = np.divide(test_scaled, X_std, out=test_scaled)
np.random.shuffle(test_scaled)

In [46]:
calculate_fid(model, train, test)

2022-08-25 10:35:52.458916: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 402653184 exceeds 10% of free system memory.


10.573613168012267

In [70]:
calculate_fid(model, test[:3000], train[:3000])

10.425674584971656

In [49]:
calculate_fid(model, train[:len(train)//2], train[len(train)//2:])

0.05398092000254107

In [50]:
calculate_fid(model, test[:len(test)//2], test[len(test)//2:])

0.014454785789405237

In [68]:
calculate_fid(model, generations, train[:len(generations)])

1.0992568604468582

In [69]:
calculate_fid(model, generations, test[:len(generations)])

7.924173031070388